<a href="https://colab.research.google.com/github/joimb9064/master_thesis/blob/main/Genetic_Tabu_search_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>